# Project Title

**Authors:** Brandon Sienkiewicz, Ilan Haskel, Zach Cherna


## Overview

Insert overview

## Business Problem

Insert business problem

## Data Understanding

The provided data contains information on movies from various websites. These data sets include general information about the movies as well as more specific data such as rating, production cost, and total gross.

In [93]:
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3
import seaborn as sns

conn = sqlite3.connect('zippedData/im.db')
%matplotlib inline

In [178]:
bom_df = pd.read_csv('zippedData/bom.movie_gross.csv.gz')
rtm_df = pd.read_csv('zippedData/rt.movie_info.tsv.gz', sep = "\t")
rtr_df = pd.read_csv('zippedData/rt.reviews.tsv.gz', sep = "\t", encoding = 'latin1')
tmdb_df = pd.read_csv('zippedData/tmdb.movies.csv.gz')
tn_df = pd.read_csv('zippedData/tn.movie_budgets.csv.gz')
imbd_df = pd.read_sql("""
SELECT *
FROM movie_basics b
JOIN movie_ratings r
ON b.movie_id = r.movie_id
""", conn)

## Data Preparation and Modelling

### Data Preparation 1

### Data Modelling 1

### Data Preparation 2

### Data Modelling 2

### Data Preparation 3

### Data Modelling 3

## Evaluation

## Conclusion